# Extract cube data from XBRL reports

In [ ]:
# @title Filter eForm Reports (most recent 650 entries) - leave blank for all
form_types = "1, 3Q_Gas, 3Q_Electric" # @param {type:"string"}
import requests
from bs4 import BeautifulSoup
import re

url = 'https://ecollection.ferc.gov/api/rssfeed'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'xml')

items = soup.find_all('item')
for item in items:
    title = item.find('title').text
    pub_date = item.find('pubDate').text


    description = item.find('description').text
    description_soup = BeautifulSoup(description, 'html.parser')

    # Find the link ending with .html
    html_link_tag = description_soup.find('a', href=lambda href: href and href.endswith('.html'))
    html_link = html_link_tag['href'] if html_link_tag else 'No HTML link found'

    # Extract the Form Type from the HTML URL
    form_type_match = re.search(r'-Form_(.*?)-2', html_link)
    form_type = form_type_match.group(1) if form_type_match else 'No Form Type found'

    # Filter by Form Type
    if not form_types.strip() or form_type in [ft.strip() for ft in form_types.split(',')]:
        print(f"Entity Name: {title}")
        print(f"HTML URL: {html_link}")
        print(f"Form Type: {form_type} Published: {pub_date}")
        print("-" * 100)

Entity Name: West Texas Gas Utility, LLC
HTML URL: https://eCollection.ferc.gov/api/DownloadDocument/366187/3?filename=273401-C000652-Form_3Q_Gas-2025-Q2_273401.html
Form Type: 3Q_Gas Published: Tue, 09 Sep 2025 17:47:09 -0400
----------------------------------------------------------------------------------------------------
Entity Name: Pine Needle LNG Company, LLC
HTML URL: https://eCollection.ferc.gov/api/DownloadDocument/366124/3?filename=273369-C000650-Form_3Q_Gas-2025-Q2_273369.html
Form Type: 3Q_Gas Published: Mon, 08 Sep 2025 15:50:14 -0400
----------------------------------------------------------------------------------------------------
Entity Name: Jackson Prairie
HTML URL: https://eCollection.ferc.gov/api/DownloadDocument/366112/3?filename=273364-C011209-Form_3Q_Gas-2025-Q2_273364.html
Form Type: 3Q_Gas Published: Mon, 08 Sep 2025 14:31:13 -0400
----------------------------------------------------------------------------------------------------
Entity Name: Scout V Hugoto

## Run this cell to install the Arelle processor and XULE plugin to extract data

In [ ]:
# @title
import os, shutil, sys, site, platform
print('Please wait while Arelle, XULE and some helper packages are installed. \nA XULE version message appears below when the environment is ready.')

# In this example, Arelle and aniso are required to use XULE - get Arelle release details from GitHub (https://github.com/Arelle/arelle/releases).
# Use %pip -q install git+https://git@github.com/Arelle/arelle.git@master to use Arelle's development release
%pip -q install Arelle-release==2.37.6
%pip -q install aniso8601==9.0.1

# 1) locate Arelle's plugin directory (do not modify this location); remove temp and xuledir if they exist
plugindir = site.getsitepackages()[0] + '/arelle/plugin/'
xuledir = plugindir + 'xule/'
xodeldir = plugindir + 'xodel/'
serializerdir = plugindir + 'serializer/'
SimpleXBRLModeldir = plugindir + 'SimpleXBRLModel/'
temp = plugindir + 'temp/'
tempxule = temp + 'plugin/xule/'
tempxodel = temp + 'plugin/xodel/'
tempserializer = temp + 'plugin/serializer/'
tempSimpleXBRLModel = temp + 'plugin/SimpleXBRLModel/'
if os.path.exists(temp):
        if os.path.isdir(temp):
            os.remove(plugindir + 'semanticHash.py')
            shutil.rmtree(xuledir)
            shutil.rmtree(xodeldir)
            shutil.rmtree(serializerdir)
            shutil.rmtree(SimpleXBRLModeldir)
            shutil.rmtree(temp)
else: ''
os.chdir(plugindir)

# 2) copy a XULE release from GitHub (https://github.com/xbrlus/xule/releases) to plugin directory
!git clone --quiet --depth=1 --branch 30050 --single-branch https://github.com/xbrlus/xule.git temp &> /dev/null
shutil.move(temp + 'plugin/semanticHash.py', plugindir)
shutil.move(tempxule, xuledir)
shutil.move(tempxodel, xodeldir)
shutil.move(tempserializer, serializerdir)
shutil.move(tempSimpleXBRLModel, SimpleXBRLModeldir)

# 3) confirm XULE (change -v to -h and re-run to see help contents for Arelle and XULE)
!arelleCmdLine --plugins 'xule' -v
print('Arelle is installed and XULE is ready')

Please wait while Arelle, XULE and some helper packages are installed. 
A XULE version message appears below when the environment is ready.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.1 MB/s eta 0:00:00
[info] Activation of plug-in XBRL rule processor (xule) successful, version Check version using Tools->Xule->Version on the GUI or --xule-version on the command line. - xule 
[info] Xule version: 3.0.30050 - 
Arelle is installed and XULE is ready


## Edit and run this cell to save variables for save location, file name, output and initial report

In [3]:
location = "/content/"
name = "extracted"
format = ".xlsx"
reportfile = location + "sample.html" #download an XBRL report, rename it as sample.html and uploaded to Colab's storage directory
# reportfile = "https://www.sec.gov/Archives/edgar/data/0001467858/000146785825000032/gm-20241231.htm" # alternatively, this is inline XBRL submitted to the SEC
print('data will be saved to ' + location + name + format)


data will be saved to /content/extracted.xlsx


## Run this cell to save a file called 'extract-data.xule' with code to extract data from a report

In [ ]:
# @title
xule_file = '''
constant $filedir = '%s'
constant $filename = '%s'
constant $fileformat = '%s'
constant $extracted = $filedir + $filename + $fileformat

// Create

output-attribute file-location
output-attribute file-content
output-attribute file-append

output cube-data

// Identify all the cubes in the instance document

$all-cubes = taxonomy().cubes

for $cube in $all-cubes
    if $cube.cube-concept.name.local-name == 'AccumulatedProvisionForDepreciationTable'
        skip
    else
    $cube-facts = $cube.facts
    $dict = dict(
                list($cube.cube-concept.name.local-name,
                list(for $fact in $cube-facts
                    list($fact.entity.id, $fact.concept.label.text, $fact.concept.name, $fact.period.start, $fact.period.end, $fact.unit.string, $fact.dimensions.join(', ','='), $fact)
                )
                )
    )

$dict

// To write to .json instead,
// change 'spreadsheet' to 'json'
// on the line below and
// update the format variable

file-content $rule-value.to-spreadsheet
file-location $extracted
file-append true
'''
import datetime
from tzlocal import get_localzone
local_tz = get_localzone()
current_time = datetime.datetime.now(local_tz)
formatted_time = current_time.strftime('%H:%M:%S %Z on %b %d, %Y')
with open('extract-data.xule', mode='w') as file:
    file.write(xule_file % (location, name, format))
print('extract-data.xule saved ' + formatted_time)

extract-data.xule saved 11:39:51 UTC on Sep 11, 2025


## Run this cell to (1)compile the XULE expression as a .zip and (2)execute to extract data from the report to file output.

In [ ]:
FILE_NAME = 'extract-data.xule'
ZIP_NAME = 'extract-data.zip'
LOG_LOCATION = location + name + '-log.xml'
print('Start compiling XULE.\n\n')

# compile XULE into .zip
!arelleCmdLine --plugins "xule" --xule-compile $FILE_NAME \
--xule-rule-set $ZIP_NAME --logFormat="[%(messageCode)s] %(message)s"

# copy XULE and .zip to /content/ directory (a Colab-specific location)
shutil.copy(FILE_NAME, location+FILE_NAME)
shutil.copy(ZIP_NAME, location+ZIP_NAME)
print('\nEnd compiling and start analyzing ' + reportfile + '\n\n' )

# run .zip to create output
!arelleCmdLine --plugins "xule|inlineXbrlDocumentSet" \
--xule-rule-set $ZIP_NAME \
-v -f $reportfile \
--xule-time .000 --xule-debug --noCertificateCheck \
--logFormat="[%(messageCode)s] %(message)s" # this option writes log to a file instead: --logFile $LOG_LOCATION
print('\n\nData extracted to ' + location + name + format )

Start compiling XULE.


[info] Activation of plug-in XBRL rule processor (xule) successful, version Check version using Tools->Xule->Version on the GUI or --xule-version on the command line.
[info] Xule version: 3.0.30050
2025-09-11T11:06:56.280111: extract-data.xule parse start
2025-09-11T11:06:56.936746: extract-data.xule parse end. Took 0:00:00.656635
2025-09-11T11:06:56.936968: extract-data.xule ast start
2025-09-11T11:06:56.937236: extract-data.xule ast end. Took 0:00:00.000268
2025-09-11T11:06:56.937271: post parse start
2025-09-11T11:06:56.939134: post parse end. Took 0:00:00.001863
2025-09-11T11:06:56.941306: Parsing finished. Took 0:00:00.661740

End compiling and start analyzing /content/sample.html


[info] Activation of plug-in XBRL rule processor (xule) successful, version Check version using Tools->Xule->Version on the GUI or --xule-version on the command line.
[info] Activation of plug-in Inline XBRL Document Set successful, version 1.1.
[info] Xule version: 3.0.30050
[l